## Online Newton Step 
1. Инициализировать $x_0 \in \mathcal{K}, \epsilon, \gamma < 1$
2. $A_0 = \epsilon I_n$ 
3. Для $t = 0 \dots T - 1:$
4. $A_t = A_{t-1} - \frac{r_t r_t^T}{(r_t^T x_t)^2}$
5. $y_{t + 1} = x_t + \frac{1}{\gamma} A_t^{-1} \frac{r_t}{r_t^T x_t}$
6. $x_{t+1} = \prod_{\mathcal{K}} (y_{t+1})$

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv('/kaggle/input/daily-historical-stock-prices-1970-2018/historical_stock_prices.csv')

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
data.shape

In [ ]:
len(data.ticker.unique())

In [ ]:
data2 = data[['ticker', 'close', 'date']].copy()

In [ ]:
mask = (data2['date'] >= '2014-01-01') & (data2['date'] < '2018-01-01')
data3 = data2.loc[mask]

In [ ]:
len(data3.ticker.unique())

In [ ]:
data3['return'] = data3['close'] / data3.shift(1)['close']

In [ ]:
df = data3[['ticker', 'date', 'return']].pivot(index=['ticker'], columns=['date'], values=['return']).iloc[:, 1:]
df = df[~df.isna().any(axis=1)].iloc[:10]

In [ ]:
# https://gist.github.com/daien/1272551/edd95a6154106f8e28209a1c7964623ef8397246
def proj_on_simplex(v):
    n, = v.shape  
    if v.sum() == 1 and np.alltrue(v >= 0):
        return v
    u = np.sort(v)[::-1]
    cssv = np.cumsum(u)
    rho = np.nonzero(u * np.arange(1, n + 1) > (cssv - 1))[0][-1]
    theta = float(cssv[rho] - 1) / (rho + 1)
    w = (v - theta).clip(min=0)
    return w

In [ ]:
def newton(x, eps, gamma, returns, T=None): 
    A = np.eye(len(x)) * eps
    if T is None:
        T = returns.shape[0]
    for t in range(T):
        grad = - returns[t] / (returns[t].T.dot(x))
        A = A - np.dot(grad[:, np.newaxis], grad[np.newaxis, :])
        y = x - np.dot(np.linalg.inv(A), grad) / gamma
        x = proj_on_simplex(y)
    return x

In [ ]:
x = np.array([0.1] * 10)
eps = 1e-2
T = 80
gamma = 1 / 8 / T ** 0.5
print(newton(x, eps, gamma, df.to_numpy().T))

In [ ]:
def grad_descent(x, eta, returns, T=None): 
    if T is None:
        T = returns.shape[0]
    for t in range(T):
        grad = - returns[t] / (returns[t].T.dot(x))
        y = x - eta * grad
        x = proj_on_simplex(y)
    return x

In [ ]:
x = np.array([0.1] * 10)
eta = 0.01
print(grad_descent(x, eta, df.to_numpy().T))